# Project Group - 25

Members: Yun-An LIN (Jackie), Rohan Menezes, John Kuttikat, Muhammad Rizki Ziarieputra (Kiki), Ian Trout 

Student numbers: 5841682, 5850908, 5765382, 5848113, 5851483

# Research Objective

*Requires data modeling and quantitative research in Transport, Infrastructure & Logistics*

Vessel time spent in ports by country before and during COVID--an analysis by ship category showing the impacts of COVID

# Contribution Statement

*Be specific. Some of the tasks can be coding (expect everyone to do this), background research, conceptualisation, visualisation, data analysis, data modelling*

**Author 1**: coding, background research, conceptualisation

**Author 2**:coding, visualisation

**Author 3**: coding, data analysis
    
**Author 4**:coding, data modelling

**Author 5**: coding, visualisation

# Data Used

----Covid data (https://data.humdata.org/dataset/coronavirus-covid-19-cases-and-deaths) 

----Port data (https://unctadstat.unctad.org/wds/TableViewer/tableView.aspx?ReportId=170027)

----total cargo loaded/unloaded by region from 1970 to 2020 (https://www.kaggle.com/datasets/illiaparfeniuk/maritime-trading-volumes)

----Total amount of goods imported and exported by ship per EU country(https://ec.europa.eu/eurostat/databrowser/view/ttr00009/default/map?lang=en)

# Data Pipeline

take only the last 6 months of each year (limitation of the maritime data): 
    
convert the maritime data:
    
    1) to a common volume 
    
    2) calculate the average volume for all cargo types 
    
    3) consolidate the data into regions of the world. 

convert COVID cases: 
    1) calculate the average vaccination cases per country that has reported it 
    2) calculate the average COVID cases per country for the last 6 months of every year (July to December) 
    
Analyze port call times for 2018, 2019, compared to 2020 to see the difference with COVID.

---calculate the differences 

Compare the 2020 and 2021 port call times to see if improvements have been made or if port calls are still slow. 

Visually show the change in port call times by region of the world by year. 



# Part I

In [1]:
import pandas as pd
import chardet
from plotly.offline import init_notebook_mode
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import json
import itertools
import geopandas as gpd

init_notebook_mode(connected=True)
pio.renderers.default = "plotly_mimetype+notebook"

In [4]:
file_path = r"/Users/iantrout/TIL6022-group_project/Data/Maritime data/US_PortCalls_S_ST202209220924_v1.csv"
with open(file_path, 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))
result

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [5]:
# Importing and Touching-up the Data

df_ports = pd.read_csv(file_path,encoding='utf-8')
df_ports['Period Label'] = df_ports['Period Label'].str.replace('   ','-')
df_ports = df_ports.drop(columns=['Period', 'Frequency', 'Frequency Label', 'Economy', 
                                      'CommercialMarket', 'Median time in port (days) Footnote',
                                      'Average age of vessels Footnote', 'Average size (GT) of vessels Footnote',
                                      'Maximum size (GT) of vessels Footnote', 'Average cargo carrying capacity (dwt) per vessel Footnote',
                                      'Maximum cargo carrying capacity (dwt) of vessels Footnote','Average container carrying capacity (TEU) per container ship Footnote',
                                      'Maximum container carrying capacity (TEU) of container ships Footnote'])
df_ports.rename(columns = {'Economy Label': 'Location', 'CommercialMarket Label': 'Vessel Type', }, inplace=True)
df_ports.head()

,Period Label,Year,Location,Vessel Type,Median time in port (days),Average age of vessels,Average size (GT) of vessels,Maximum size (GT) of vessels,Average cargo carrying capacity (dwt) per vessel,Maximum cargo carrying capacity (dwt) of vessels,Average container carrying capacity (TEU) per container ship,Maximum container carrying capacity (TEU) of container ships
0,S1-2018,2018,World,All ships,0.97,18,15222,234006,24074.0,441561.0,3526.0,21413.0
1,S1-2018,2018,World,Passenger ships,NaN,21,8978,228081,NaN,NaN,NaN,NaN
2,S1-2018,2018,World,Liquid bulk carriers,0.94,13,15470,234006,26871.0,441561.0,NaN,NaN
3,S1-2018,2018,World,Container ships,0.69,13,38405,217673,NaN,NaN,3526.0,21413.0
4,S1-2018,2018,World,Dry breakbulk carriers,1.12,19,5455,91784,7413.0,138743.0,NaN,NaN
